#### Model used: nli-deberta-v3-base

In [1]:
!pip install protobuf==3.20.3 -q

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
onnx 1.20.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
ray 2.52.1 requires click!=8.3.*,>=7.0, but you have click 8.3.1 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.

In [2]:
reference_answers = {

    "Q1": (
        "Object-Oriented Programming (OOP) is a programming paradigm that organizes software "
        "design around objects that combine data and behavior. Unlike procedural programming, "
        "which focuses on functions operating on data, OOP models real-world entities as objects. "
        "The four core principles of OOP are encapsulation, abstraction, inheritance, and polymorphism. "
        "Encapsulation hides internal data and exposes it through controlled interfaces, abstraction "
        "focuses on essential features, inheritance enables code reuse through class hierarchies, and "
        "polymorphism allows different objects to respond differently to the same method. A real-world "
        "example of encapsulation is a bank account, where balance details are accessed only through "
        "methods like deposit and withdraw. OOP may not be ideal for small or performance-critical "
        "systems such as embedded software."
    ),

    "Q2": (
        "Arrays and linked lists are fundamental data structures used to store collections of elements. "
        "Arrays store elements in contiguous memory locations, allowing constant-time access using an "
        "index, but insertions and deletions are costly because elements may need to be shifted. Linked "
        "lists store elements in non-contiguous memory locations, where each node contains data and a "
        "reference to the next node, making access slower due to traversal but allowing efficient "
        "insertions and deletions. Arrays provide O(1) access and O(n) insertion and deletion in the "
        "worst case, while linked lists provide O(n) access and O(1) insertion or deletion when the "
        "position is known. A real-world example where linked lists are useful is a music playlist."
    ),

    "Q3": (
        "Time complexity measures how an algorithm’s running time grows as the input size increases, "
        "and it is important for comparing algorithms independently of hardware. Big-O notation "
        "expresses the upper bound of an algorithm’s growth rate. Linear time O(n) occurs when an "
        "algorithm processes each element once, logarithmic time O(log n) appears in algorithms like "
        "binary search, and quadratic time O(n^2) occurs in algorithms with nested loops such as "
        "bubble sort. Worst-case analysis is preferred because it guarantees performance limits under "
        "all possible input conditions."
    ),

    "Q4": (
        "Compiled programming languages translate source code into machine code before execution, "
        "producing an executable file, while interpreted languages execute code line by line at runtime "
        "using an interpreter. Examples of compiled languages include C and C++, while Python and "
        "JavaScript are commonly interpreted. Just-In-Time compilation, used in languages like Java, "
        "combines both approaches by compiling frequently executed code segments at runtime to improve "
        "performance. Interpreted languages are not always slower than compiled ones due to modern "
        "runtime optimizations."
    ),

    "Q5": (
        "A process is an independent program in execution with its own memory space, while a thread "
        "is a lightweight unit of execution within a process that shares memory with other threads. "
        "Context switching is the mechanism by which the operating system saves the state of a running "
        "task and restores another, allowing multitasking and efficient CPU utilization. Processes do "
        "not share memory by default, making them safer but heavier, whereas threads share memory, "
        "enabling faster communication but requiring synchronization. Multithreading improves "
        "performance in applications such as web servers handling multiple client requests."
    ),

    "Q6": (
        "Database normalization is the process of organizing data to reduce redundancy and improve "
        "integrity. First normal form ensures atomic values with no repeating groups. Second normal "
        "form removes partial dependencies so that non-key attributes depend on the entire primary key. "
        "Third normal form removes transitive dependencies so that non-key attributes depend only on "
        "the primary key. Excessive normalization can lead to complex queries and performance overhead "
        "due to multiple joins. Denormalization is often used in read-heavy systems such as analytics "
        "and reporting platforms."
    ),

    "Q7": (
        "Machine Learning enables systems to learn patterns from data rather than relying on manually "
        "written rules. Supervised learning uses labeled data such as spam email classification, "
        "unsupervised learning works with unlabeled data for tasks like customer segmentation, and "
        "reinforcement learning learns through rewards and penalties such as training agents to play "
        "games. Key challenges include data dependency, bias, lack of interpretability, and difficulty "
        "in generalizing to unseen data."
    ),

    "Q8": (
        "Artificial Intelligence is the field of creating systems capable of performing tasks that "
        "require human-like intelligence, such as reasoning and decision-making. Machine Learning is a "
        "subset of AI that allows systems to learn from data. Not all AI systems use Machine Learning; "
        "some rely on rule-based logic. Deep learning is a subset of Machine Learning that uses multi-"
        "layer neural networks. An example of non-ML AI is a rule-based expert system that uses "
        "predefined rules to make decisions."
    ),

    "Q9": (
        "When a user enters a URL, the browser resolves the domain name to an IP address using DNS. It "
        "then establishes a connection with the server using TCP through a three-way handshake. If "
        "HTTPS is used, a TLS handshake encrypts the communication. The browser sends a request, the "
        "server responds with encrypted data, and the browser decrypts and renders the webpage. HTTPS "
        "ensures secure communication."
    ),

    "Q10": (
        "The Software Development Lifecycle is a structured process that includes requirement analysis, "
        "design, implementation, testing, deployment, and maintenance. The waterfall model follows "
        "these phases sequentially and is suitable when requirements are stable. Agile development is "
        "iterative and incremental, emphasizing flexibility, continuous feedback, and frequent "
        "releases. Agile is preferred for dynamic projects, while waterfall suits regulated or "
        "contract-based environments."
    )

}


In [3]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd

In [5]:
df= pd.read_csv("/kaggle/input/scores/scores.csv")

In [6]:
model_name= "cross-encoder/nli-deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

2026-01-05 15:36:28.001957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767627388.204386      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767627388.258800      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767627388.749436      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767627388.749472      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767627388.749475      55 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [8]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [9]:
def nli_scores(premise, hypothesis):
    inputs= tokenizer(premise, hypothesis, return_tensors="pt", truncation=True, padding=True, max_length=512) #output returned as pytorch tensor!!
    inputs= {k: v.to(device) for k,v in inputs.items()}

    with torch.no_grad():
        outputs= model(**inputs) #unpacking

    probs= F.softmax(outputs.logits, dim=1)[0]

    return {
        "contradiction": probs[0].item(),
        "neutral": probs[1].item(),
        "entailment": probs[2].item()
    }

# index 0 → contradiction 
# index 1 → neutral 
# index 2 → entailment

In [13]:
entailment_scores = []
contradiction_scores = []

for _, row in df.iterrows():
    qid = row["question_id"]
    student_answer = row["answer"]
    reference_answer = reference_answers[qid]

    if not student_answer.strip():
        entailment_scores.append(0.0)
        contradiction_scores.append(0.0)
        continue

    scores = nli_scores(reference_answer, student_answer)

    entailment_scores.append(scores["entailment"] * 100)
    contradiction_scores.append(scores["contradiction"] * 100)

In [14]:
df["nli_entailment_score"] = entailment_scores
df["nli_contradiction_score"] = contradiction_scores

In [16]:
df.to_csv("/kaggle/working/scores.csv")